### Project 02: Create a GUI Notebook Program

Julia, Giovanni, Nayab, Vivek

Project 2 will adapt the procedural code we have been working on in INST326_SimpleGUI_Note_Form_IO.ipynb to create an OOP version of the program using three classes.

    A Notebook or MainWindow class
    A Form or TopWindow class
    A Note class

The MainWindow class is a subclass of Tkinter’s tk.Tk class and thus inherits its attributes and methods, while allowing us to customize the new window objects to our needs. These new window objects will represent “notebooks” or collections of notes, and allow us to work with those notes. (I have named it MainWindow because this class definition could be used to create any kind of main window in Tkinter. We are using it to represent notebooks in this application, but you might use it for other purposes in onther applications.)


The TopWindow class creates a new top window in Tkinter, which is essentially a form for entering the title, text, links, and tags for our note. When we submit the note, this “form” object has a method that creates the note’s metadata and a new Note object. That note object is appended to the list of all notes, which is an attribute of the notebook (MainWindow) class.
The Note class creates note objects that contain the  title, text, links, tags, and metadata for each note.

For Project 02 you will:  

    1. Create one notebook or MainWindow object  
    2. Create several (at least 3) ‘real’ notes for your final submission. By ‘real’ I mean actual notes about python that are useful to you. Print them in the cell at the bottom of the notebook.
    3. Save those notes to a single .txt, .csv, or .json file (your choice of format).  
    4. Retrieve those notes and 
    5. Display representations of them as either labels or buttons in the  main window (remember how we displayed cards in project 01). These representations are not whole notes. Rather, they are object representations of the notes.  
    6. When they are clicked the whole note pops up in a new window - either the form window or a ‘read-only’ version of the form window.



#### Complete your code in the cell below

The code cell below contains the imports you will need; the top level structure for the three classes to get you started; and the execution code at the bottom. 

In [29]:
# imports
import tkinter as tk
from tkinter import ttk
from tkinter import scrolledtext
import datetime # one module for working with dates and times
import json
from functools import partial
import uuid

class MainWindow(tk.Tk):
    def __init__(self):  #initialize the main window
        super().__init__() # initialize it as a tkinter window
        
        self.geometry("600x400") # set the default window size
        self.title('Notebook') #set the default window title
        # self.notebook = [] # initialize an attribute named 'notebook' as an empty list
        #^said by TA its okay to not use as long as no issues, said might be used in the future and when creating the key she didn't really use the variable
        self.notes = []
        self.icons = []        

        # add additionallines to the __init__() function
        self.new_button = tk.Button(self, text = 'New', command = self.new_note)
        self.new_button.grid(row = 0, column = 0)

        self.open_button = tk.Button(self, text = 'Open Notebook', command = self.open_notebook)
        self.open_button.grid(row = 0, column = 1)

        self.save_button = tk.Button(self, text = 'Save Notebook', command = self.save_notebook)
        self.save_button.grid(row = 0, column = 2)

    def open_note(self, note):
        self.noteform = NoteForm(self, self.notes, note)
        return

    def new_note(self):
        NoteForm(self, self.notes, None)
        return

    def refresh_icons(self):
        # clear existing icons
        for button in self.icons:
            button.destroy()
        self.icons = []

        # draw icons for all notes
        add_row = 1
        for note in self.notes:
            self.note_button = tk.Button(self, text = note.new_note["title"], command = partial(self.open_note, note))
            self.note_button.grid(row = add_row, column = 0)
            add_row = add_row + 1
            self.icons.append(self.note_button)

        return
    
    def open_notebook(self):
        self.notes = []
        with open("json_notes", "r") as jsonfile:
            read_notes = json.load(jsonfile)
        print(read_notes)
        # convert read_notes to self.notes
        for note in read_notes:
            note_object = MakeNote(note)
            self.notes.append(note_object)

        self.refresh_icons()

        return 

    def save_notebook(self):
        saved_notes = []
        for note in self.notes:
            saved_notes.append(note.new_note)
        json_notes = json.dumps(saved_notes)
        # print(json_notes)
        with open("json_notes", "w") as jsonfile:
            jsonfile.write(json_notes)

        

class NoteForm(tk.Toplevel):
    def __init__(self, master, notes, note): # initialize the new object
        super().__init__(master) # initialize it as a toplevel window
        self.master = master
        self.notes = notes
        self.is_new_note = True
        if (note): # opening an existing note
            self.note = note.new_note
            self.is_new_note = False
        else: # create new note with default values
            self.note = {
                "title": "New note title",
                "text": "lorem",
                "code_snippets": "Code Snippets",
                "author": "Author",
                "link": 'If there is a link with this note enter it here. Else, enter "None"',
                "tags": 'Enter #tags, separated by commas',
                "time": str(datetime.datetime.now().time()),
                "date": str(datetime.datetime.now().date()),
                "timezone": str(datetime.datetime.now().astimezone().tzinfo),
                "Note ID" : str(uuid.uuid4()),
            }

        # create our note title entry field
        self.note_title = tk.Entry(self, width=80)
        self.note_title.grid(padx=10, pady=10, row=1, column=1, sticky='w')
        self.note_title.insert(0, self.note["title"]) # adds default text (useful during development)

    # create our note text field
        self.note_text = tk.Text(self, height=10, width=80,)
        self.note_text.grid(padx=10, pady=10, row=2, column=1)
        self.note_text.insert('1.0', self.note["text"]) # adds default text (useful during development)
        
        self.note_codeSnippet = tk.Text(self, height=10, width=80,)
        self.note_codeSnippet.grid(padx=10, pady=10, row=3, column=1)
        self.note_codeSnippet.insert('1.0', self.note["code_snippets"])

        self.submit_button = tk.Button(self, text = 'Submit Note', command = self.submit)
        self.submit_button.grid(row = 0, column = 0)

        self.note_author = tk.Entry(self, width=60)
        self.note_author.grid(padx=10, pady=10, row=4, column=1, sticky='w')
        self.note_author.insert(0, self.note['author'])

        self.note_link = tk.Entry(self, width=60)
        self.note_link.grid(padx=10, pady=10, row=5, column=1, sticky='w')
        self.note_link.insert(0, self.note['link']) # adds default text (useful during development)
        
        # create our note tags entry field
        self.note_tags = tk.Entry(self, width=60)
        self.note_tags.grid(padx=10, pady=10, row=6, column=1, sticky='w')
        self.note_tags.insert(0, self.note['tags']) # adds default text (useful during development)
    
        # dates and times can be tricky. consult the datetime documentation.
        # self.current_time = self.note["time"]
        # self.current_date = self.note["date"]
        # self.local_timezone = self.note["timezone"]

        self.note_ID = self.note["Note ID"]

    def submit(self):
        # add lines to the submit method
        new_note = {
            "title": self.note_title.get(),
            "text": self.note_text.get('1.0', 'end'),
            "code_snippets": self.note_codeSnippet.get('1.0', 'end'),
            "author": self.note_author.get(),
            "link": self.note_link.get(),
            "tags": self.note_tags.get(),
            "date": str(datetime.datetime.now().date()),
            "time": str(datetime.datetime.now().time()),
            "timezone": str(datetime.datetime.now().astimezone().tzinfo),
            "Note ID": self.note_ID
            }
        if self.is_new_note:
            self.notes.append(MakeNote(new_note))
        else:
            for note in self.notes:
                if self.note_ID == note.new_note["Note ID"]:
                    note.update(new_note)

        self.master.refresh_icons()
        self.destroy()
        return 

class MakeNote():
    def __init__(self, note_dict):
        self.new_note = note_dict
        self.history = []
        # print(self.new_note)
    
    def get(self):
        return self.new_note

    def update(self, note_dict):
        self.history.append(self.new_note)
        self.new_note = note_dict

    def __str__(self):
        return f'{self.new_note["title"]}{self.new_note["text"]}{self.new_note["link"]}{self.new_note["tags"]}{self.new_note["date"]}{self.new_note["time"]}{self.new_note["timezone"]}'


# main execution

if __name__ == '__main__':
    
    main_window = MainWindow() # this creates a notebook / main window called main_window. You may change the name if you want

    main_window.mainloop()

#### Print your three notes below

In [30]:
with open("json_notes", "r") as jsonfile:
    read_notes = json.load(jsonfile)

print(read_notes)
#remember to submit json files along with project

[{'title': 'sooo', 'text': 'lorem\n\n', 'code_snippets': 'Code Snippets\n\n', 'author': 'Author', 'link': 'If there is a link with this note enter it here. Else, enter "None"', 'tags': 'Enter #tags, separated by commas', 'date': '2024-04-23', 'time': '01:51:09.158304', 'timezone': 'Eastern Daylight Time', 'Note ID': '69087ab4-98ef-439a-b922-4ccbfb71131d'}, {'title': 'New note title', 'text': 'whyyy\n', 'code_snippets': 'Code Snippets\n', 'author': 'Author', 'link': 'If there is a link with this note enter it here. Else, enter "None"', 'tags': 'Enter #tags, separated by commas', 'date': '2024-04-23', 'time': '01:51:16.457686', 'timezone': 'Eastern Daylight Time', 'Note ID': 'ae803e40-2d01-47c9-af42-62536402643d'}]
